In [ ]:
from keras.datasets import reuters
import numpy as np

In [ ]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

In [ ]:
#print(train_data.shape)
#print(test_data.shape)
#print(train_data[0])
#print(train_data[1])

print(np.max(np.max(train_data)))

In [ ]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [ ]:
def print_decoded_newswire(index):
    decoded_newswire = " ".join([reverse_word_index.get(i - 3, "?") for i in train_data[index]])
    print(decoded_newswire)

In [ ]:
for i in range(2):
    print_decoded_newswire(i)

In [ ]:
print(train_labels[0:10])
print(np.max(train_labels))

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

In [ ]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(x_train[0])
print(np.mean(x_train[0]))

In [ ]:
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1
    return results

In [ ]:
one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

In [ ]:
print(one_hot_train_labels.shape)
print(one_hot_test_labels.shape)
print(one_hot_train_labels[0])
print(np.argmax(one_hot_train_labels[0]))

In [ ]:
from keras.utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

In [ ]:
print(one_hot_train_labels.shape)
print(one_hot_test_labels.shape)
print(one_hot_train_labels[0])
print(np.argmax(one_hot_train_labels[0]))

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation="relu", input_shape=(10000,)))
model.add(layers.Dense(4, activation="relu"))
model.add(layers.Dense(46, activation="softmax"))

In [ ]:
model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

In [ ]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=9,
    batch_size=512,
    validation_data=(x_val, y_val)
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, val_loss)
plt.show()

In [ ]:
acc = history.history["acc"]
val_acc = history.history["val_acc"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, acc, val_acc)
plt.show()

In [ ]:
result = model.evaluate(x_test, one_hot_test_labels)
print(result)

In [ ]:
def plot_hist(history_dict):
   fig = plt.figure(figsize=(5,4))
   f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5),sharey=True)
   
   ax1.plot(epochs,  history_dict['loss'],label="Train")
   ax1.plot(epochs,  history_dict['val_loss'],label="CV")
   ax1.set_title("Loss function development - Training set vs CV set")
   ax1.legend(loc='upper right')
   ax1.set_xlabel('Epochs')
   ax1.set_ylabel('Values')
   
   ax2.plot(epochs,  history_dict['acc'],label="Train")
   ax2.plot(epochs,  history_dict['val_acc'],label="CV")
   ax2.set_title("Accuracy development - Training set vs CV set")
   ax2.legend(loc='upper right')
   ax2.set_xlabel('Epochs')
   
   plt.suptitle('TITLE', fontsize=16)

In [ ]:
plot_hist(history.history)